<a href="https://colab.research.google.com/github/AlirezaParchami/Real-time-Character-Detection-using-Faster-RCNN/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Dependencies and Detectron2

In [ ]:
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html

In [6]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

In [ ]:
#Download Character Dataset and unzip
#!wget https://github.com/MinhasKamal/AlphabetRecognizer/files/1084725/English.Alphabet.Dataset.zip
#!unzip English.Alphabet.Dataset.zip

In [3]:
from detectron2.structures import BoxMode
from detectron2.data import MetadataCatalog, DatasetCatalog

def get_character_dict(img_dir, num):
  dir = img_dir + str(num)+ "/" + str(num) + ".json"
  json_file = os.path.join(dir)
  
  with open(json_file) as f:
    imgs_anns = json.load(f)
  
  character_dicts = []
  for idx, v in zip(imgs_anns.keys(), imgs_anns.values()):
    record = {}

    filename = os.path.join(img_dir + str(num) + "/" + v["filename"])
    height, width = cv2.imread(filename).shape[:2]

    record["file_name"] = filename
    record["image_id"] = idx
    record["height"] = height
    record["width"] = width

    annos = v["regions"]
    objs = []
    for _, anno in annos.items():
      anno = anno["shape_attributes"]
      px = anno["points_x"]
      py = anno["points_y"]
      poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
      poly = [p for x in poly for p in x]

      obj = {
          "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
          "bbox_mode": BoxMode.XYXY_ABS,
          "segmentation": [poly],
          "category_id": num - 1,
      }
      objs.append(obj)
    record["annotations"] = objs
    character_dicts.append(record)
  return character_dicts
  

for d in range(1,8):
  DatasetCatalog.register("character_train_" + str(d), lambda d=d: get_balloon_dicts("English Alphabet Dataset/", d))
  MetadataCatalog.get("character_train_" + str(d)).set(thing_classes=[str(d)])
character_metadata = MetadataCatalog.get("character_train")

Metadata(name='character_train')


TypeError: ignored

In [ ]:
#Check if the metadata is added successfully

from detectron2.utils.visualizer import Visualizer
dataset_dicts = get_character_dict("English Alphabet Dataset/", 5)
print(len(dataset_dicts))
print(len(dataset_dicts[1]))
print(dataset_dicts[1])
for d in random.sample(dataset_dicts, 3):
  print(d)
  img = cv2.imread(d["file_name"])
  visualizer = Visualizer(img[:, :, ::-1], metadata=character_metadata, scale=0.5)
  out = visualizer.draw_dataset_dict(d)
  cv2_imshow(out.get_image()[:, :, ::-1])